In [35]:
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import dataloader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

train = pickle.load( open( "/content/drive/MyDrive/uba/deep learning/train.pkl", "rb" ) )
train_labels = pickle.load( open( "/content/drive/MyDrive/uba/deep learning/train_label.pkl", "rb" ) )

print(train.shape)
print(np.unique(train_labels))

import os
import pandas as pd
from torchvision.io import read_image

tr = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])

class YourDataset(Dataset):

    def __init__(self, X_Train, Y_Train, transform=None):
        self.X_Train = X_Train
        self.Y_Train = Y_Train
        self.transform = transform

    def __len__(self):
        return len(self.X_Train)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        x = self.X_Train[idx]
        y = self.Y_Train[idx]

        if self.transform:
            x = self.transform(x)
            y = self.transform(y)

        return x, y


file = open('/content/drive/MyDrive/uba/deep learning/train.pkl', 'rb')
X_train = pickle.load(file)
X_train = X_train[..., np.newaxis]
X_train = X_train.swapaxes(1, 3)

file.close()

file = open('/content/drive/MyDrive/uba/deep learning/train_label.pkl', 'rb')
Y_train = pickle.load(file)
file.close()

train_dataset = YourDataset(X_train, Y_train)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=0)
file = open('/content/drive/MyDrive/uba/deep learning/val.pkl', 'rb')
X_val = pickle.load(file)
X_val = X_val[..., np.newaxis]
X_val = X_val.swapaxes(1, 3)

file.close()

file = open('/content/drive/MyDrive/uba/deep learning/val_label.pkl', 'rb')
Y_val = pickle.load(file)
file.close()

val_dataset = YourDataset(X_val, Y_val)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=True, num_workers=0)

file = open('/content/drive/MyDrive/uba/deep learning/test.pkl', 'rb')
X_test = pickle.load(file)
X_test = X_test[..., np.newaxis]
X_test = X_test.swapaxes(1, 3)

file.close()

file = open('/content/drive/MyDrive/uba/deep learning/test_label.pkl', 'rb')
Y_test = pickle.load(file)
file.close()

test_dataset = YourDataset(X_test, Y_test)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=True, num_workers=0)

print("train:", len(train_dataset), "\nval:", len(val_dataset), "\ntest:", len(test_dataset))

class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=3,stride=1,padding="same")
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=3,stride=1,padding="same")
        self.fc1 = nn.Linear(16 * 7 * 7, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Model()

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

for epoch in range(30):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs.float())
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0

print('Finished Training')

(56000, 28, 28)
[0 1 2 3 4 5 6 7 8 9]
train: 56000 
val: 7000 
test: 7000
[1,   200] loss: 0.532
[2,   200] loss: 0.122
[3,   200] loss: 0.088
[4,   200] loss: 0.072
[5,   200] loss: 0.061
[6,   200] loss: 0.052
[7,   200] loss: 0.046
[8,   200] loss: 0.042
[9,   200] loss: 0.037
[10,   200] loss: 0.034
[11,   200] loss: 0.029
[12,   200] loss: 0.027
[13,   200] loss: 0.025
[14,   200] loss: 0.022
[15,   200] loss: 0.020
[16,   200] loss: 0.019
[17,   200] loss: 0.016
[18,   200] loss: 0.016
[19,   200] loss: 0.014
[20,   200] loss: 0.013
[21,   200] loss: 0.011
[22,   200] loss: 0.010
[23,   200] loss: 0.009
[24,   200] loss: 0.009
[25,   200] loss: 0.008
[26,   200] loss: 0.006
[27,   200] loss: 0.006
[28,   200] loss: 0.006
[29,   200] loss: 0.005
[30,   200] loss: 0.004
Finished Training


In [38]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = model(images.float())
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 98 %
